In [ ]:
import numpy as np

import pygsti
from pygsti.noisymodelpacks import noisymodelpack as nmp
from pygsti.objects.operation import ComposedOp, StochasticNoiseOp

In [ ]:
pspec = pygsti.objects.ProcessorSpec(10, ['Gi', 'Gxpi', 'Gypi', 'Gcnot'])

In [ ]:
mdl = nmp.create_pauli_stochastic_noise_model(pspec, ['Gi', ('Gi', 9), 'Gcnot'])

In [ ]:
def print_stochastic_gates(mdl):
    for lbl, op in mdl.operation_blks['gates'].items():
        print(f'Gate {lbl}\n{op}')
        if isinstance(op, ComposedOp) and isinstance(op.factorops[1], StochasticNoiseOp):
            noise_op = op.factorops[1]
            rates = noise_op._params_to_rates(noise_op.params)
            print(f'Total strength: {sum(rates)}\nRates: {rates}\n\n')

print_stochastic_gates(mdl)

In [ ]:
rand_state = np.random.RandomState(2021)

# Create stochastic noise model with:
# slightly noisy idle, bad idle on qubit 9, and noisy CNOT gates
# CNOT noise allows up to weight 2 (i.e. correlated) and different each qubit
# Allow random total noise and random distribution among channels
noisy_mdl = nmp.sample_noise_model(mdl,
                                   max_total_strengths={
                                        'Gi': 0.01,
                                        ('Gi', 9): 0.5,
                                        'Gcnot': 0.1},
                                   max_weight=2,
                                   rand_state=rand_state)

In [ ]:
print_stochastic_gates(noisy_mdl)

In [ ]:
# Let's try with only weight 1 on CNOT and make it symmetric
noisy_mdl2 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     max_weight=1,  # from 2
     symmetric_strengths=True, # from False
     rand_state=rand_state)

In [ ]:
print_stochastic_gates(noisy_mdl2)

In [ ]:
# Let's generate a model with deterministic total strength
noisy_mdl3 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     random_total_strength=False,   # from True
     rand_state=rand_state)

In [ ]:
print_stochastic_gates(noisy_mdl3)

In [ ]:
# Or maybe random total strength but depolarizing
noisy_mdl4 = nmp.sample_noise_model(mdl,
     max_total_strengths={
         'Gi': 0.01,
         ('Gi', 9): 0.5,
         'Gcnot': 0.1},
     random_relative_strengths=False, # from True
     rand_state=rand_state)

In [ ]:
print_stochastic_gates(noisy_mdl4)